<a href="https://colab.research.google.com/github/qazalmehrban/Medical-Segmentation/blob/ghazal/numpy_array.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from skimage.transform import resize
import numpy as np
import nibabel as nib# for reading nifty files
import os 




In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
IMG_HEIGHT, IMG_WIDTH =300,300

In [5]:
def gen(filter = True):
  batch_i = []
  batch_m =[]
  samples = '/content/drive/MyDrive/samples/'
  patients = next(os.walk(samples))[2]
  patients.sort()
  labels = '/content/drive/MyDrive/segmentation/'
  segmentations =next(os.walk(labels))[2]
  segmentations.sort()
  for i,j in zip(patients,segmentations):
    Images = nib.load((samples + i))
    im = np.array(Images.dataobj)
    Masks = nib.load((labels + j))    
    Masks = np.array(Masks.dataobj)
    img_resize = resize(im,
               (IMG_HEIGHT, IMG_WIDTH),
             mode = 'constant',
              anti_aliasing=True,
               preserve_range=True)
    mask_resize = resize(Masks,
               (IMG_HEIGHT, IMG_WIDTH),
             mode = 'constant',
              anti_aliasing=True,
               preserve_range=True)
    for k in range(im.shape[2]):
      if filter:
        if np.all(mask_resize[:,:,k]==0):
          continue
        else :
          im1 = np.expand_dims(img_resize[:,:,k],2)
          ma = np.expand_dims(mask_resize[:,:,k],2)
          batch_i.append(im1)
          batch_m.append(ma)
      else :


        im1 = np.expand_dims(img_resize[:,:,k],2)
        ma = np.expand_dims(mask_resize[:,:,k],2)
        batch_i.append(im1)
        
        batch_m.append(ma)
  batch_image = np.stack(batch_i)
  batch_mask = np.stack(batch_m)


  return batch_image, batch_mask


In [6]:
total_image , total_mask = gen()

In [7]:
total_image.shape, total_mask.shape

((608, 300, 300, 1), (608, 300, 300, 1))

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((total_image, total_mask))

In [9]:
dataset.element_spec

(TensorSpec(shape=(300, 300, 1), dtype=tf.float64, name=None),
 TensorSpec(shape=(300, 300, 1), dtype=tf.float64, name=None))